In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
import os
from sklearn.preprocessing import MinMaxScaler

from pyproj import CRS
import pydeck as pdk
import matplotlib.font_manager as fm
from sklearn.preprocessing import LabelEncoder
fontprop = fm.FontProperties(fname='../font/NanumGothic.ttf', size=10)

/opt/app-root/lib/python3.6/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


### data load

In [2]:
%%time
#딕셔너리에 각 파일 순서대로 load
file_list = os.listdir('./data')
file_list = file_list[:-1]
files = {}
for i,file in enumerate(file_list):
    try :
        files[i] = pd.read_csv(f'./data/{file}')
    except : 
        files[i] = gpd.read_file(f'./data/{file}')

<string>:2: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.


CPU times: user 14.1 s, sys: 177 ms, total: 14.2 s
Wall time: 15 s


In [3]:
mapbox_key = 'pk.eyJ1IjoibGFpcjA4MjYiLCJhIjoiY2tkcGoxcnRzMDZvODJxbXk0MWhlcWN2aSJ9.5-yjt_SUq4w5JII7CvD4cA'
map_center = [127.07377,37.14885] #오산시 중심
view_states = pdk.ViewState(longitude=map_center[0],
                          latitude = map_center[1],
                           zoom = 12)

In [53]:
def add_coordinates(data):
    '''geodataframe에서 중점 coordinates로 컬럼 생성
    '''
    from shapely.geometry import Polygon,MultiLineString,LineString,MultiPolygon
    #coord_center
    temp = data.copy()
    coord_center = temp.geometry.apply(lambda geo : [geo.centroid.x, geo.centroid.y])
    temp['coord_center'] = coord_center
    
    #coordinates type별로 조금씩 차이가 있음
    type_mls = type(MultiLineString())
    type_ls = type(LineString())
    type_poly = type(Polygon())
    type_mpoly = type(MultiPolygon())
    def get_coordinates(x):
        if type(x) == type_mls: 
            lon,lat = x[0].xy
        elif type(x) == type_ls:
            lon,lat = x.xy
        elif type(x) == type_poly:
            lon,lat = x.exterior.xy
        else :
            lon, lat = x[0].exterior.xy        
        return [[x, y] for x, y in zip(lon, lat)]
    
    temp['coordinates'] = temp.geometry.apply(lambda geo : get_coordinates(geo))
    
    return temp

def add_minmax(data,col):
    '''minmax scale 추가
    '''
    from sklearn.preprocessing import MinMaxScaler
    scaler = MinMaxScaler()
    temp = data.copy()
    values = scaler.fit_transform(temp[col].values.reshape(-1,1))
    temp['minmax_value'] = values
    
    return temp

def merge_to_grid(value_df,merged_df, col_name,lon = 'lon', lat = 'lat',):
    '''value_df가 merge_df포함되는 갯수를 체크하고 merge진행. 
    GeoDataFrame인 경우 point 생성부분은 pass
    ----
    return 
    value_df가 merge_df가 join된 df
    '''
    type_gpd = type(gpd.GeoDataFrame())
    if 'gid' in value_df.columns:
        value_df = value_df.rename(columns={'gid':'gid1'})
    
    if type(value_df) != type_gpd:
        value_df['geometry'] = value_df.apply(lambda x : make_Point(x[lon],x[lat]),axis= 1)
        value_df = gpd.GeoDataFrame(value_df)
        value_df.crs = CRS('epsg:4326')

    joined = gpd.sjoin(merged_df,value_df, how = 'left')
    if col_name not in joined.columns:
        joined.loc[~(joined['index_right'].isna()),col_name] = 1
    joined[col_name] = joined[col_name].fillna(0)

    joined = joined.groupby('gid')[col_name].sum().reset_index()
    
    return joined

def draw_hist(data,col,title):
    '''data를 넣으면 histogram을 그려주고, 0 or NaN 없는 df 리턴
    
    '''
    temp = data.copy()
    temp = add_coordinates(temp[~((temp[col]==0)|(temp[col].isna()))])
    plt.hist(temp[col])
    plt.title(title,fontproperties = fontprop, fontsize= 15)
    plt.box(False)
    plt.show()
    return temp

def make_Point(x,y):
    from shapely.geometry import Point
    '''x,y좌표값이 있으면, 합쳐서 Point로 만듬
    '''
    return Point(x,y)

def move_df(main_df,merged_df, s_time, l_time, col_name):
    '''
    main_df(df) : 유동인구 df
    merged_df(df) : 합칠 df, ex) 어린이교통사고_df
    s_time(int) : 유동인구 시작시간
    l_time(int) : 유동인구 끝 시간
    col_name(string) : 저장 colmn 이름
    
    -----
    return 
    merged_df에 분할시간대 유동인구의 평균값 컬럼생성
    '''
    temp = main_df.copy()
    #stack형태로 변경
    melted = pd.melt(temp, id_vars = ['STD_YM','lon','lat'],var_name = 'time')
    melted.time = melted.time.apply(lambda x : int(x[-2:]))
    
    #필요한 시간대로 분할
    grouped = melted.groupby(['lon','lat','time'])['value'].sum().reset_index()
    grouped_cut = grouped[(grouped.time>=s_time)&(grouped.time<=l_time)]
    
    #point생성

    grouped_cut = grouped_cut.groupby(['lon','lat'])['value'].sum().reset_index()
    grouped_cut['geometry'] = grouped_cut.apply(lambda x : make_Point(x['lon'],x['lat']),axis= 1)
    
    #geo dataframe화 
    geo_df = gpd.GeoDataFrame(grouped_cut)
    geo_df.crs = CRS('epsg:4326')
    
    #merge_df와 결합
    joined = gpd.sjoin(merged_df,geo_df, how = 'left', op = 'intersects')
    joined['value'] = joined['value'].fillna(0)
    joined_g = joined.groupby('gid')['value'].mean()
    joined_g.name = col_name
    joined_g = joined_g.reset_index()
#     moved = files[1].merge(joined_g,how = 'left')
    
    return joined_g

In [39]:
#주정차단속 merge
joined = merge_to_grid(files[0],files[1],'주정차단속수',lon = '단속위치_경도',lat = '단속위치_위도')
grid_df = files[1].merge(joined, how= 'left')

In [42]:
#자동차대수 merge
joined = merge_to_grid(files[2],files[1],'car_cnt',lon = '단속위치_경도',lat = '단속위치_위도')
grid_df = grid_df.merge(joined, how= 'left')
grid_df = grid_df.rename(columns = {'car_cnt':'자동차대수'})

In [48]:
#총인구 merge
joined = merge_to_grid(files[3],files[1],'val')
grid_df = grid_df.merge(joined, how= 'left')
grid_df = grid_df.rename(columns = {'val':'총인구수'})

In [49]:
#유소년인구 merge
joined = merge_to_grid(files[4],files[1],'val')
grid_df = grid_df.merge(joined, how= 'left')
grid_df = grid_df.rename(columns = {'val':'유소년인구수'})

In [58]:
#생산가능인구 merge
joined = merge_to_grid(files[5],files[1],'val')
grid_df = grid_df.merge(joined, how= 'left')
grid_df = grid_df.rename(columns = {'val':'생산가능인구인구수'})

In [50]:
#고령인구 merge
joined = merge_to_grid(files[6],files[1],'val')
grid_df = grid_df.merge(joined, how= 'left')
grid_df = grid_df.rename(columns = {'val':'고령인구수'})

In [59]:
#유동인구 merge
joined = move_df(files[7],files[1],7,9,'유동인구7_9')
grid_df = grid_df.merge(joined, how= 'left')

joined = move_df(files[7],files[1],16,18,'유동인구16_18')
grid_df = grid_df.merge(joined, how= 'left')

In [61]:
#어린이보호구역 merge
joined = merge_to_grid(files[8],files[1],'CCTV설치대수',lon = '보호구역_경도',lat = '보호구역_위도')
grid_df = grid_df.merge(joined, how= 'left')

In [62]:
#유치원수
joined = merge_to_grid(files[12],files[1], '유치원수', lon = '시설위치_경도',lat = '시설위치_위도')
grid_df = grid_df.merge(joined,how = 'left')

In [63]:
#무인카메라수 
joined = merge_to_grid(files[14],files[1], '무인단속카메라수', lon = '설치위치_경도',lat = '설치위치_위도')
grid_df = grid_df.merge(joined,how = 'left')

In [64]:
#표지판수
joined = merge_to_grid(files[15],files[1], '표지판수')
grid_df = grid_df.merge(joined,how = 'left')

In [67]:
#횡단보도수
joined = merge_to_grid(files[16],files[1],'횡단보도수')
grid_df = grid_df.merge(joined,how = 'left')

In [68]:
#과속방지턱
joined = merge_to_grid(files[17],files[1],'과속방지턱수',lon = '설치위치_경도',lat = '설치위치_위도')
grid_df = grid_df.merge(joined,how = 'left')

In [ ]:
#신호등 
joined = merge_to_grid(files[18],files[1],'신호등수')
grid_df = grid_df.merge(joined,how = 'left')

In [70]:
#CCTV
joined = merge_to_grid(files[19].iloc[:-1,:],files[1],'CCTV수',lon = '설치위치_경도',lat= '설치위치_위도')
grid_df = grid_df.merge(joined,how = 'left')

/opt/app-root/lib/python3.6/site-packages/ipykernel_launcher.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [71]:
#인도/자전거도로
bike_df = files[20][files[20].BYYN == 'BYC001']
walk_df = files[20][files[20].BYYN == 'BYC002']

joined = merge_to_grid(walk_df,files[1],'인도수')
grid_df = grid_df.merge(joined,how = 'left')

joined = merge_to_grid(bike_df,files[1],'자전거도로수')
grid_df = grid_df.merge(joined,how = 'left')

In [72]:
#버스정류장
joined = merge_to_grid(files[21],files[1],'버스정류장수',lon = '정류장 위치_경도',lat = '정류장 위치_위도')
grid_df = grid_df.merge(joined,how = 'left')

In [ ]:
#추정교통량
s_time = 7
l_time = 9

#시간범위 설정
files[23] = files[23].rename(columns = {'상세도로망_LinkID':'link_id'})
files[23].link_id = files[23].astype(str)
f23 = files[23][['link_id','시간적범위','전체 추정교통량', '승용차 추정교통량', '버스 추정교통량', '화물차 추정교통량']]
f23 = f23[f23['시간적범위'] != '전일']
f23.시간적범위 = f23.시간적범위.astype(int)
melted = pd.melt(f23, id_vars = ['link_id','시간적범위'])

f23_time_cut = f23[(f23.시간적범위>=s_time)&(f23.시간적범위<=l_time)]
f23_time_cut = f23_time_cut.groupby('link_id')['전체 추정교통량', '승용차 추정교통량', '버스 추정교통량', '화물차 추정교통량'].sum().reset_index()

#컬럼이름 재설정
rename_ls = []
for i, col in enumerate(f23_time_cut.columns):
    if i> 0:
        new_name = col + f'{s_time}_{l_time}'
        rename_ls.append(new_name)
    else :
        new_name = col
        rename_ls.append(new_name)

f23_time_cut.columns = rename_ls

#intersect and merge
road_merged = files[22][['link_id','geometry']].merge(f23_time_cut,how = 'left')
temp = gpd.sjoin(grid_df[['gid','geometry']],road_merged)
temp = temp.groupby('gid')[rename_ls].mean().reset_index()
grid_df = grid_df.merge(temp,how = 'left')
grid_df = grid_df.fillna(0)

In [ ]:
#혼잡빈도강도

#intersect and merge
files[24]['link_id'] = files[24]['상세도로망_LinkID'].apply(lambda x : str(x)[:-2])
f24_gt = files[24][files[24]['시간적범위'] == '전일']
f24_gt = f24_gt.groupby(['link_id'])['혼잡빈도강도'].sum().reset_index()

road_merged = files[22][['link_id','geometry']].merge(f24_gt,how = 'left')
temp = gpd.sjoin(grid_df[['gid','geometry']],road_merged)
temp = temp.groupby('gid')['혼잡빈도강도'].mean().reset_index()

grid_df = grid_df.merge(temp,how = 'left')
grid_df = grid_df.fillna(0)

In [ ]:
#혼잡시간강도 `
#intersect and merge
files[25]['link_id'] = files[25]['상세도로망_LinkID'].apply(lambda x : str(x)[:-2])
f25_gt = files[25][files[25]['시간적범위'] == '전일']
f25_gt = f25_gt.groupby(['link_id'])['혼잡시간강도'].sum().reset_index()

road_merged = files[22][['link_id','geometry']].merge(f25_gt,how = 'left')
temp = gpd.sjoin(grid_df[['gid','geometry']],road_merged)
temp = temp.groupby('gid')['혼잡시간강도'].mean().reset_index()

grid_df = grid_df.merge(temp,how = 'left')
grid_df = grid_df.fillna(0)

In [ ]:
#체육시설수
joined = merge_to_grid(files[26],files[1],'체육시설수', lon = '설치위치_경도',lat = '설치위치_위도')
grid_df = grid_df.merge(joined,how = 'left')

In [ ]:
#건물개수
joined = merge_to_grid(files[27],files[1],'건물개수')
grid_df = grid_df.merge(joined,how = 'left')

In [ ]:
#층수 (반복실행시 rename으로 인해 중복컬럼생성됨)
joined = merge_to_grid(files[27],files[1],'floor')
grid_df = grid_df.merge(joined,how = 'left')
grid_df = grid_df.rename(columns={'floor':'총층수'})

In [ ]:
#연면적 (반복실행시 rename으로 인해 중복컬럼생성됨)
joined = merge_to_grid(files[28],files[1],'val')
grid_df = grid_df.merge(joined,how = 'left')
grid_df = grid_df.rename(columns={'val':'건물연면적'})

In [ ]:
#학원수
joined = merge_to_grid(files[29],files[1],'학원수', lon = '시설위치_경도', lat = '시설위치_위도')
grid_df = grid_df.merge(joined,how = 'left')

In [69]:
grid_df

,gid,accident_cnt,geometry,주정차단속수,자동차대수,자동차대수,유소년인구수,총인구수,총인구수,유소년인구수,...,moved7_9,유동인구7_9,생산가능인구인구수,유동인구16_18,CCTV설치대수,유치원수,무인단속카메라수,표지판수,횡단보도수,과속방지턱수
0,다사551085,0,"MULTIPOLYGON (((126.99422 37.17418, 126.99421 ...",0.0,0,0.0,0.0,0.0,0.0,0.0,...,0.000,0.000,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0
1,다사551086,0,"MULTIPOLYGON (((126.99421 37.17508, 126.99420 ...",0.0,0,0.0,0.0,0.0,0.0,0.0,...,2.035,2.035,0.0,2.385,0.0,0.0,0.0,0.0,0.0,0.0
2,다사551087,0,"MULTIPOLYGON (((126.99420 37.17599, 126.99420 ...",0.0,0,0.0,0.0,0.0,0.0,0.0,...,21.130,21.130,0.0,20.305,0.0,0.0,0.0,0.0,0.0,0.0
3,다사551088,0,"MULTIPOLYGON (((126.99420 37.17689, 126.99419 ...",0.0,0,0.0,0.0,0.0,0.0,0.0,...,13.930,13.930,0.0,14.790,0.0,0.0,0.0,0.0,0.0,0.0
4,다사552085,0,"MULTIPOLYGON (((126.99534 37.17419, 126.99534 ...",0.0,0,0.0,0.0,0.0,0.0,0.0,...,1.060,1.060,0.0,0.920,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4505,다사646052,0,"MULTIPOLYGON (((127.10138 37.14484, 127.10138 ...",0.0,0,0.0,0.0,0.0,0.0,0.0,...,0.000,0.000,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0
4506,다사646053,0,"MULTIPOLYGON (((127.10138 37.14575, 127.10137 ...",0.0,0,0.0,0.0,0.0,0.0,0.0,...,0.000,0.000,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0
4507,다사647050,0,"MULTIPOLYGON (((127.10252 37.14305, 127.10251 ...",0.0,0,0.0,0.0,0.0,0.0,0.0,...,0.000,0.000,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0
4508,다사647051,0,"MULTIPOLYGON (((127.10251 37.14395, 127.10251 ...",0.0,0,0.0,0.0,0.0,0.0,0.0,...,0.000,0.000,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0
